**Generating a New Episode of Peaky Blinders with AI21Lab's Jurassic-2 Model on Amazon SageMaker!**

The objective of this notebook is to generate a new episode for Peaky Blinders, encompassing a title, summary.

The workflow of the process follows these steps:

Create Peaky Blinders dataset, comprising summaries from the series' first six seasons.
Eliminating the episodes without title.
Building few-shot examples.
Generate the new episode.

**Installing and importing libraries**

In [331]:
!pip install -U ai21[AWS]
!pip install numpy
!pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [332]:
import ai21
import numpy as np
import pandas as pd

**API key**

You need an API key for AI21 Studio for this notebook. You can find in in your account in AI21 Labs website.

In [333]:
# Put your API key from AI21 studio account
ai21.api_key = "ADD_YOUR_API_KEY_HERE"
assert ai21.api_key != "", "You must provide an API key!"    

**Loading the dataset**

In [334]:
df = pd.read_csv("PeakyBlinders.csv")
df.head(20)

,Season,Episode,Title,Summary
0,1,1,NaN,"In 1919 following the Great War, the Peaky Bli..."
1,1,2,NaN,"Tommy, Arthur and their brother John Shelby me..."
2,1,3,NaN,Ada and Freddie marry; Polly gives them money ...
3,1,4,NaN,Tommy makes his business legitimate by obtaini...
4,1,5,NaN,"Believing Tommy betrayed Freddie, Ada does not..."
5,1,6,NaN,Inspector Campbell confirms to Churchill that ...
6,2,1,NaN,Grace shoots Inspector Campbell and leaves. Tw...
7,2,2,NaN,Campbell visits Tommy in the hospital and reve...
8,2,3,NaN,"The gang hire the Digbeth Kid, a neighbourhood..."
9,2,4,NaN,Tommy meets with his Irish handlers and discov...


**Dropping episodes without titles and create subset**

In [335]:
dropped_df = df.dropna()
dropped_df.head(15)

,Season,Episode,Title,Summary
18,4,1,The Noose,"As Arthur, John, Michael and Polly prepare to ..."
19,4,2,Heathens,Michael survives his injuries but John is kill...
20,4,3,Blackbird,The Italians launch another attack on the Peak...
21,4,4,Dangerous,Arthur goes to the house of the family whose s...
22,4,5,The Duel,Tommy leads Changretta’s gang to Artillery Squ...
23,4,6,The Company,"Tommy watches the boxing match with Arthur, wh..."
24,5,1,Black Tuesday,Polly is dead. Arthur receives an intimidating...
25,5,2,Black Cats,Tommy discovers landmines and a scarecrow dres...
26,5,3,Strategy,Aberama is taken to a gypsy camp by Polly and ...
27,5,4,The Loop,"Tommy and Arthur meet with Brilliant Chang, a ..."


In [336]:
SEPARATOR = '\n########\n'
SUBSET_SIZE = 7
SEED = 23

In [337]:
# Sample a subset of the episodes for few-shot generations
df_subset = dropped_df.sample(n=SUBSET_SIZE, random_state=SEED)
df_subset

,Season,Episode,Title,Summary
18,4,1,The Noose,"As Arthur, John, Michael and Polly prepare to ..."
28,5,5,The Shock,"Tommy, Polly and Arthur tend to Linda's gunsho..."
20,4,3,Blackbird,The Italians launch another attack on the Peak...
19,4,2,Heathens,Michael survives his injuries but John is kill...
22,4,5,The Duel,Tommy leads Changretta’s gang to Artillery Squ...
23,4,6,The Company,"Tommy watches the boxing match with Arthur, wh..."
31,6,2,Black Shirt,"Tommy, still a Labour MP, returns to England a..."


In [338]:
# Sample a subset of the episodes for few-shot generations
random_df_subset = df_subset.sample(n=SUBSET_SIZE, random_state=SEED)

**Building Few-shot example**

At this point, I had to make a sample for a few-shot scenario. First, I make a prompt based on the existing titles title. This prompt helps me generate prompt summaries. Then, I create an example that includes both the prompt and its completion. This example helps with training and generating the desired results. Next, I build a prefix using a few examples to provide more context for the prompt. Finally, I combine the prefix of examples with a new title query to create a few-shot prompt that allows me to generate relevant summaries.

In [339]:
def create_prompt(title):
    prompt = "Generate a Peaky Blinders episode based on the title.\n"
    prompt += "Episode Title: "
    prompt += title
    prompt += "\nEpisode Summary:\n"
    
    return prompt


def generate_summary_from_single_example(title, summary):
    example = create_prompt(title)
    example += summary
    
    return example

SUMMARY_FEW_SHOT_PREFIX = SEPARATOR.join(
    random_df_subset.apply(lambda x: generate_summary_from_single_example(x["Title"], x["Summary"]), axis=1)
)

In [340]:
def create_summary_fewshot_prompt(title):
    return SUMMARY_FEW_SHOT_PREFIX + SEPARATOR + create_prompt_for_summary(title)

In [341]:
prompt_summary = create_summary_fewshot_prompt(generated_title)
print(prompt_summary)

Generate a Peaky Blinders episode based on the title.
Episode Title: Black Shirt
Episode Summary:
Tommy, still a Labour MP, returns to England and is informed Ruby is recovering. He has a seizure in the bathroom and Lizzie urges him to see a doctor, but he refuses. After Jack Nelson expresses interest to meet the fascists, Tommy sees an opportunity in an alliance with Captain Swing. He visits Alfie and announces the death of his uncle in the Cotton Club in New York at the hands of Jack Nelson's gang. After a tense meeting with Mosley and his mistress Diana Mitford, Tommy meets Jack Nelson and agrees to feed information about England's politics to Nelson's gang in exchange for Tommy being able to sell his opium in Boston. Tommy is informed that Ruby is sick again and decides to contact Esme.
########
Generate a Peaky Blinders episode based on the title.
Episode Title: The Duel
Episode Summary:
Tommy leads Changretta’s gang to Artillery Square. Tommy has prepared for the battle ahead of 

**Let's generate the new episode of Peaky Blinders**

Now it is time to generate the new episode. Here I make a request to j2-mid with my summary prompt, limiting the maximum number of tokens to 200. 

In [342]:
response = ai21.Completion.execute(model="j2-mid",
                                   prompt=prompt_summary,
                                   maxTokens=200,
                                   stopSequences=[SEPARATOR],
                                   temperature=0.7
                                  )

generated_episode = response['completions'][0]['data']['text']
print(f"Title: {generated_title}")
print("----------------")
print(f"Summary: {generated_episode}")

Title: The Betrayal
----------------
Summary: The Peaky Blinders are betrayed by Esme, who has become a pawn of Mosley. Mosley and his men hold Billy Kimber, Michael's friend and lover, hostage. Tommy visits Mosley and demands that he release Billy. Mosley refuses, and Tommy shoots Mosley's men. Mosley escapes and Tommy is arrested. Esme visits Tommy in prison and tells him that Billy is dead. Tommy then breaks out of prison and goes to Scotland, where he meets with Jessie and tells her that he knows that she betrayed him to Mosley. Esme visits Tommy in Scotland and tells him that she has turned against Mosley and that he must return to Birmingham to face justice. Tommy agrees and returns to Birmingham with Esme, where Mosley has arranged for him to be tried by a military tribunal. The trial begins, and Tommy is acquitted of all charges. Michael, Polly, and Arthur are released from prison and reunite with Tommy and Esme. The Peaky Blinders disband, and Tommy and Esme go to America to s